In [3]:
import requests
import pandas as pd
import pprint
import os
from dotenv import load_dotenv

In [33]:
#Lectura e importacion de datos
id_ind = {'3108001001' : 'Porcentaje analfabetismo',
          '1005000038' : 'Promedio escolaridad',
          '6200205239' : 'Poblacion con educacion basica',
          '6200205241' : 'Poblacion con bachillerato',
          '6200205242' : 'Poblacion con estudios superiores',
          '6207019020' : 'Porcentaje sin escolaridad',
          '1004000001' : 'Derechohabientes'
          }

id_estado = {
    "Aguascalientes": "01",
    "Baja California": "02",
    "Baja California Sur": "03",
    "Campeche": "04",
    "Coahuila de Zaragoza": "05",
    "Colima": "06",
    "Chiapas": "07",
    "Chihuahua": "08",
    "Ciudad de México": "09",
    "Durango": "10",
    "Guanajuato": "11",
    "Guerrero": "12",
    "Hidalgo": "13",
    "Jalisco": "14",
    "México": "15",
    "Michoacán de Ocampo": "16",
    "Morelos": "17",
    "Nayarit": "18",
    "Nuevo León": "19",
    "Oaxaca": "20",
    "Puebla": "21",
    "Querétaro": "22",
    "Quintana Roo": "23",
    "San Luis Potosí": "24",
    "Sinaloa": "25",
    "Sonora": "26",
    "Tabasco": "27",
    "Tamaulipas": "28",
    "Tlaxcala": "29",
    "Veracruz": "30",
    "Yucatán": "31",
    "Zacatecas": "32"
}

load_dotenv()
token_inegi = os.getenv("token_inegi")

indicadores = (",".join(list(id_ind.keys()))).strip()
valores = []

for estado in id_estado:
    inegi_url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{indicadores}/es/{id_estado[estado]}/false/BISE/2.0/{token_inegi}?type=json"
    #inegi_url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{indicadores}/es/070000{id_estado[estado]}/false/BISE/2.0/{token_inegi}?type=json"
    response = requests.get(inegi_url)
    data = response.json()
    for indicador in data['Series']: 
        for observacion in indicador['OBSERVATIONS']:
            indicador_actual = indicador['INDICADOR']
            fecha = observacion['TIME_PERIOD']
            valor = observacion['OBS_VALUE']
            valores.append({"id_estado": id_estado[estado], "estado": estado, "indicador": indicador_actual, "indicador_nombre": id_ind[indicador_actual], "año" : fecha, "valor" : valor})

#print(pprint.pp(valores))
        

In [45]:
#Creacion del dataframe
df_edu = pd.DataFrame(valores)
df_edu['valor'] = pd.to_numeric(df_edu['valor'], errors='coerce')
df_edu_wide = df_edu.pivot_table(index=['id_estado','estado', 'año'], columns = 'indicador_nombre', values = 'valor').reset_index()
df_edu_wide.columns = ['id_estado','estado', 'fecha', 'derhab', 'pob_bac', 'pob_edbas', 'pob_edsup', 'porc_analf', 'porc_sined', 'promedio_ed']
df_edu_wide.head()

,id_estado,estado,fecha,derhab,pob_bac,pob_edbas,pob_edsup,porc_analf,porc_sined,promedio_ed
0,01,Aguascalientes,1910,NaN,NaN,NaN,NaN,64.300000,NaN,NaN
1,01,Aguascalientes,1921,NaN,NaN,NaN,NaN,53.800000,NaN,NaN
2,01,Aguascalientes,1930,NaN,NaN,NaN,NaN,56.100000,NaN,NaN
3,01,Aguascalientes,1940,NaN,NaN,NaN,NaN,39.800000,NaN,NaN
4,01,Aguascalientes,1950,NaN,NaN,NaN,NaN,31.000000,NaN,NaN
5,01,Aguascalientes,1960,NaN,NaN,NaN,NaN,22.500000,NaN,NaN
6,01,Aguascalientes,1970,NaN,NaN,NaN,NaN,16.600000,NaN,NaN
7,01,Aguascalientes,1980,NaN,NaN,NaN,NaN,10.900000,NaN,NaN
8,01,Aguascalientes,1990,NaN,27312.0,NaN,24581.0,7.100000,NaN,NaN
9,01,Aguascalientes,1995,NaN,NaN,NaN,NaN,5.600000,NaN,NaN


In [47]:
#Guardado de dataframe
nombre_carpeta = '../.data/raw/'
nombre_archivo = 'educacionysalud.csv'

if not os.path.exists(nombre_carpeta):
    os.makedirs(nombre_carpeta)
    print("Se creó la carpeta .data/raw/")

ruta_guardado = os.path.join(nombre_carpeta, nombre_archivo)
df_edu_wide.to_csv(ruta_guardado, index=False, encoding='utf-8')
print("Archivo con dataframe de educacion y salud creado")


Archivo con dataframe de educacion y salud creado


In [53]:
df_edu_wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_estado    479 non-null    object 
 1   estado       479 non-null    object 
 2   fecha        479 non-null    object 
 3   derhab       128 non-null    float64
 4   pob_bac      96 non-null     float64
 5   pob_edbas    64 non-null     float64
 6   pob_edsup    96 non-null     float64
 7   porc_analf   447 non-null    float64
 8   porc_sined   64 non-null     float64
 9   promedio_ed  160 non-null    float64
dtypes: float64(7), object(3)
memory usage: 37.6+ KB


In [55]:
#Revisamos si hay valores nulos
df_edu_wide.isna().sum()

id_estado        0
estado           0
fecha            0
derhab         351
pob_bac        383
pob_edbas      415
pob_edsup      383
porc_analf      32
porc_sined     415
promedio_ed    319
dtype: int64

In [112]:
#Limpieza de valores nulos
#df_edu_drop = df_edu.dropna(subset = ['valor'])
#df_edu_drop.isna().sum()

indicador_nombre    0
año                 0
valor               0
dtype: int64